### Question 1

In [7]:
def trim_chip_seq(file_path, out_file_path):
    
    content = []
    with open(filepath)as f:

        trimmed_file = open(out_file_path, "w") 
        
        ct = 0
        for line in f:
            content.append(line)
            ct += 1

            if ct == 100:
                break

    trimmed_file.writelines(content)
    trimmed_file.close()
    
    return 

### Question 2

In [55]:
phased_snps = pd.read_csv("provided_data/chromosome_1_phased_first_5000_snps.vcf", delimiter='\t')
phased_snps.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMP0001,...,SAMP1054,SAMP1055,SAMP1056,SAMP1057,SAMP1058,SAMP1059,SAMP1060,SAMP1061,SAMP1062,SAMP1063
0,chr1,3000185,rs585444580,G,T,.,PASS,.,GT,0|0,...,0|0,1|0,0|0,0|0,0|1,1|1,0|1,0|0,1|1,0|0
1,chr1,3000325,rs586608136,G,T,.,PASS,.,GT,1|1,...,1|1,0|1,1|1,1|1,1|0,0|0,1|0,1|1,0|0,1|1
2,chr1,3001278,rs581216520,G,A,.,PASS,.,GT,1|1,...,1|1,0|1,1|1,1|1,1|0,0|0,1|0,1|1,0|0,1|1
3,chr1,3001490,rs31521921,C,A,.,PASS,.,GT,0|0,...,0|0,1|0,0|0,0|0,0|1,1|1,0|1,0|0,1|1,0|0
4,chr1,3004940,rs578487906,C,T,.,PASS,.,GT,0|0,...,0|0,1|0,0|0,0|0,0|1,1|1,0|1,0|0,1|1,0|0


In [52]:
def split_phased_snps(phased_snps):
    """Split phased SNP data into sequential pair of maternal and paternal SNPs.

    Arguments:
        phased_snps (pd.DataFrame): a (num_snps x (9 + num_samples)) dataframe loaded from a VCF file, with first
            nine columns ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT"] and the
            following columns containing the sequenced genotypes, where each value is a string of the form

                                    "maternal_allele | paternal_allele"

            for example, an entry "0 | 1" indicates that the maternal chromosome contained the reference allele
            at that locus while the paternal chromosome contained the alternate allele.

    Returns:
        snps: a (2 x num_snps x num_samples)-dimension Numpy array of strings "0" or "1"s, where snps[0] is the
            maternal haplotype for all samples and snps[1] is the paternal haplotype for all samples
    """
    numSnps = len(phased_snps)
    numSamples = phased_snps.shape[1] - 9
    
    split_snps = np.ndarray(shape=(2, numSnps, numSamples))
    snps = phased_snps.iloc[:, 9:]
    
    for i in range(numSnps):
        for j in range(numSamples):
            
            split_snps[0, i, j] = snps.iloc[i, j][0]
            split_snps[1, i, j] = snps.iloc[i, j][-1]
    
    return split_snps


def compute_allele_frequencies(snps):
    """Compute reference and alternate allele frequencies for each SNP.

    Arguments:
        snps: a (num_snps x (2 * num_samples))-dimension Numpy array of strings "0" or "1"s, where the first num_samples
            columns are alleles from the maternal chromosome and the second num_samples columns are the corresponding
            alleles from the paternal chromosome.

    Returns:
        a 2-tuple of (snp_ref_frequency, snp_alt_frequency) representing the reference and alternate frequencies
            for each snp respectively, where snp_ref_frequency[i] + snp_alt_frequency[i] = 1
    """
    
    numSnps = len(snps)
    numSamples = len(snps[0])/2
    
    snp_ref_frequency, snp_alt_frequency = np.zeros(numSnps), np.zeros(numSnps)

    for i in range(numSnps):
        
        mat_allele_cts, pat_allele_cts = Counter(snps[i][:numSamples]), Counter(snps[i][numSamples:])
        allele_cts = {0: mat_allele_cts[0] + pat_allele_cts[0], 1: mat_allele_cts[1] + pat_allele_cts[1]}
        
        snp_ref_frequency[i] = allele_cts[0]/(2 * numSamples)
        snp_alt_frequency[i] = allele_cts[1]/(2 * numSamples)
    
    return snp_ref_frequency, snp_alt_frequency


def compute_haplotype_frequencies(snps):
    """Compute haplotype frequencies for every pair of SNPs.

    Arguments:
        snps: a (num_snps x (2 * num_samples))-dimension Numpy array of strings "0" or "1"s, where the first num_samples
            columns are alleles from the maternal chromosome and the second num_samples columns are the corresponding
            alleles from the paternal chromosome.

    Returns:
        a 4-tuple of (p_11, p_12, p_21, p_22) representing the haplotype frequencies of ("0", "0"), ("0", "1"),
            ("1", "0") and ("1", "1") for each pair respectively, where each element is a (num_snps x num_snps) Numpy
            array, and where the sum of p_11, p_12, p_21, p_22 for each pair of snps is 1.
    """
    numSnps = len(snps)
    numSamples = len(snps[0])/2
    
    p_11, p_12, p_21, p_22 = np.zeros(shape=(numSnps, numSnps)), np.zeros(shape=(numSnps, numSnps)), np.zeros(shape=(numSnps, numSnps)), np.zeros(shape=(numSnps, numSnps))
    
#     for pa in range(2):

#         ref_pa, alt_pa = None, None
#         if pa == 0:
#             ref_pa, alt_pa = 0, 1
#         if pa == 1:
#             ref_pa, alt_pa  = 1, 0
        
    for i in range(numSnps):

        snp_i_ref, snp_i_alt = snps[i][0], snps[i][1]

        for j in range(numSnps):

            snp_j_ref, snp_j_alt = snps[j][0], snps[j][1]

            haplotype_ref = list(zip(snp_i_ref, snp_j_ref))
            haplotype_alt = list(zip(snp_i_alt, snp_j_alt))
            
            haplotype_alleles = [haplotype_ref, haplotype_alt]

            for k in range(numSamples):
                
                for h in haplotype_alleles:
                    
                    if h[k] == (0.0, 0.0):
                        p_11[i, j] += 1
                    
                    if h[k] == (0.0, 1.0):
                        p_12[i, j] += 1
                        
                    if h[k] == (1.0, 0.0):
                        p_21[i, j] += 1
                    
                    if h[k] == (1.0, 1.0):
                        p_22[i, j] += 1

#                 if haplotype_ref[k] == (0.0, 0.0):
#                     p_11[i, j] += 1

#                 if haplotype_ref[k] == (0.0, 1.0):
#                     p_12[i, j] += 1

#                 if haplotype_ref[k] == (1.0, 0.0):
#                     p_21[i, j] += 1

#                 if haplotype_ref[k] == (1.0, 1.0):
#                     p_22[i, j] += 1
                
#                 if haplotype_alt[k] == (0.0, 0.0):
#                     p_11[i, j] += 1

#                 if haplotype_alt[k] == (0.0, 1.0):
#                     p_12[i, j] += 1

#                 if haplotype_alt[k] == (1.0, 0.0):
#                     p_21[i, j] += 1

#                 if haplotype_alt[k] == (1.0, 1.0):
#                     p_22[i, j] += 1
                
                        
    p_11 = p_11/(4 * numSamples)    p_12 = p_12/(4 * numSamples)

    p_12 = p_12/(4 * numSamples)
    p_21 = p_21/(4 * numSamples)
    p_22 = p_22/(4 * numSamples)
    
    haplotype_frequencies = (p_11, p_12, p_21, p_22)
    
    return haplotype_frequencies

In [ ]:
## main
phased_snps = pd.read_csv("provided_data/chromosome_19_phased_snps.vcf", delimiter="\t")
split_snps = split_phased_snps(phased_snps) ##2 x num_snps x num_samples

numSnps = len(split_snps[0])
numSamples = split_snps[0].shape[1]

## convert to 2d: num_snps x num_samples*2
snps = np.zeros(shape=(numSnps, numSamples*2))
for i in range(numSnps):
    snps[i, :num_samples] = split_snps[0][i] 
    snps[i, num_samples:] = split_snps[1][i]
    
allele_frequencies = compute_allele_frequencies(snps)
haplotype_frequencies = compute_haplotype_frequencies(snps)

### Question 3

In [40]:
import pandas as pd
import numpy as np
from collections import Counter
from scipy.stats.distributions import chi2

def split_phased_snps(phased_snps):
    """Split phased SNP data into sequential pair of maternal and paternal SNPs.

    Arguments:
        phased_snps (pd.DataFrame): a (num_snps x (9 + num_samples)) dataframe loaded from a VCF file, with first
            nine columns ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT"] and the
            following columns containing the sequenced genotypes, where each value is a string of the form

                                    "maternal_allele | paternal_allele"

            for example, an entry "0 | 1" indicates that the maternal chromosome contained the reference allele
            at that locus while the paternal chromosome contained the alternate allele.

    Returns:
        snps: a (2 x num_snps x num_samples)-dimension Numpy array of strings "0" or "1"s, where snps[0] is the
            maternal haplotype for all samples and snps[1] is the paternal haplotype for all samples
    """
    numSnps = len(phased_snps)
    numSamples = phased_snps.shape[1] - 9
    
    split_snps = np.ndarray(shape=(2, numSnps, numSamples))
    snps = phased_snps.iloc[:, 9:]
    
    for i in range(numSnps):
        for j in range(numSamples):
            
            split_snps[0, i, j] = snps.iloc[i, j][0]
            split_snps[1, i, j] = snps.iloc[i, j][-1]
    
    return split_snps

def snp_array(vcf_df, numSnps, numSamples):
    
    split_snps = np.ndarray(shape=(2, numSnps, numSamples))
    snps = vcf_df.iloc[:, 9:]
    
    for i in range(numSnps):
        for j in range(numSamples):
            
            split_snps[0, i, j] = snps.iloc[i, j][0]
            split_snps[1, i, j] = snps.iloc[i, j][-1]
    
    return split_snps


def compute_genotype_counts(split_snps):
    
    homo_ref_cts, hetero_cts, homo_alt_cts = [], [], []
    numSnps = len(split_snps[0])
    
    for i in range(numSnps):
        
        curr_genotypes = list(zip(split_snps[0][i], split_snps[1][i]))
        cts = dict(Counter(curr_genotypes))
        
        homo_ref_cts.append(cts[(0.0, 0.0)])
        homo_alt_cts.append(cts[(1.0, 1.0)])
        hetero_cts.append(cts[(0.0, 1.0)] + cts[(1.0, 0.0)])
    
    genotype_counts = (np.array(homo_ref_cts), np.array(hetero_cts), np.array(homo_alt_cts))
    
    return genotype_counts

def compute_effective_allele_frequencies(split_snps):
    
    genotype_cts = compute_genotype_counts(split_snps)
    numSnps = len(split_snps[0])
    ref_freq, alt_freq = [], []     
    
    for i in range(numSnps):
        
        total_cts = genotype_cts[0][i] + genotype_cts[1][i] + genotype_cts[2][i]

        curr_ref_freq = (2 * genotype_cts[0][i] + (genotype_cts[1][i]))/(2*total_cts)
        curr_alt_freq = (2 * genotype_cts[2][i] + (genotype_cts[1][i]))/(2*total_cts)
        
        ref_freq.append(curr_ref_freq)
        alt_freq.append(curr_alt_freq)
        
    allele_freqs = (np.array(ref_freq), np.array(alt_freq))
    
    return allele_freqs

def calculate_chi_squared_statistic(effective_allele_frequencies, genotype_counts):
    """Calculate the χ2 statistic for each SNP.

    Arguments:
        effective_allele_frequencies: a 2-tuple of (snp_ref_frequency, snp_alt_frequency) representing the reference and alternate frequencies
            for each snp respectively, where snp_ref_frequency[i] + snp_alt_frequency[i] = 1
        genotype_counts: a 3-tuple of (homozygous_reference_counts, heterozygous_counts, homozygous_alternate_counts)
            representing the homozygous reference, heterozygous and homozygous alternate counts for each snp respectively,
            where snp_ref_frequency[i] + snp_alt_frequency[i] = 1
    
    Returns:
        a num_snps-dimension Numpy array containing the χ2 statistic for each SNP.
                
    """
    
    numSnps = len(genotype_counts[0])
    numSamples = genotype_counts[0][0] + genotype_counts[1][0] + genotype_counts[2][0]
    
    exp_genotype_counts = [np.zeros(numSnps), np.zeros(numSnps), np.zeros(numSnps)]
    
    for i in range(numSnps):
        
        exp_genotype_counts[0][i] = numSamples*(effective_allele_frequencies[0][i]**2)
        exp_genotype_counts[1][i] = numSamples*(2*effective_allele_frequencies[0][i]*effective_allele_frequencies[1][i])
        exp_genotype_counts[2][i] = numSamples*(effective_allele_frequencies[1][i]**2)
    
    chi_stats = ((np.array(exp_genotype_counts) - np.array(genotype_counts))**2)/np.array(exp_genotype_counts)
    chi_stats = np.sum(chi_stats, axis = 0)
            
    return chi_stats

def detect_snps_under_selection(chi_squared_statistic, alpha=1e-2, dof=1):
    """
    Determie which SNPs violate Hardy-Weinberg equilibrium according to the χ2 statistic.

    Arguments:
        a num_snps-dimension Numpy array containing the χ2 statistic for each SNP.
    
    Returns:
        a num_snps-dimension boolean Numpy array representing whether each SNP violates or doesn't violate HW-equilibrium
    """
    
    is_selected = []
    p_vals = []
    
    for i in chi_squared_statistic:
        
        p_value = 1 - chi2.cdf(i, dof)
        p_vals.append(p_value)
        
        if p_value < alpha:
            is_selected.append(True)
        else:
            is_selected.append(False)

    return np.array(is_selected, dtype=bool), p_vals

In [41]:
## main
phased_snps = pd.read_csv("provided_data/chromosome_19_phased_snps.vcf", delimiter="\t")
split_snps = split_phased_snps(phased_snps)
genotype_counts = np.array(compute_genotype_counts(split_snps))
effective_allele_frequencies = np.array(compute_effective_allele_frequencies(split_snps))
chi_squared_statistic = calculate_chi_squared_statistic(effective_allele_frequencies, genotype_counts)
is_eq, p = detect_snps_under_selection(chi_squared_statistic, alpha=1e-2, dof=1)

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,obs_1,...,obs_1054,obs_1055,obs_1056,obs_1057,obs_1058,obs_1059,obs_1060,obs_1061,obs_1062,obs_1063
0,chr19,3127429,rs251845524,C,A,.,PASS,.,GT,1|0,...,1|0,1|1,0|0,1|1,1|1,1|0,0|1,0|0,1|1,1|1
1,chr19,3139452,rs232015029,G,A,.,PASS,.,GT,0|1,...,0|1,0|0,1|1,0|0,0|0,0|1,1|0,1|1,0|0,0|0
2,chr19,3178695,rs47851167,G,A,.,PASS,.,GT,1|0,...,1|0,1|1,0|0,1|1,1|1,1|0,0|1,0|0,1|1,1|1
3,chr19,3202439,rs262767692,A,T,.,PASS,.,GT,0|1,...,0|1,0|0,1|1,0|0,0|0,0|1,1|0,1|1,0|0,0|0
4,chr19,3203754,rs223184589,A,G,.,PASS,.,GT,0|1,...,0|1,0|0,1|1,0|0,0|0,0|1,1|0,1|1,0|0,0|0


In [44]:
genotype_counts

array([[244, 289, 244, ...,  17,  17,  17],
       [530, 530, 530, ..., 265, 265, 265],
       [289, 244, 289, ..., 781, 781, 781]])

In [45]:
effective_allele_frequencies

array([[0.47883349, 0.52116651, 0.47883349, 0.52116651, 0.52116651,
        0.52116651, 0.52116651, 0.52116651, 0.52116651, 0.52116651,
        0.52116651, 0.52116651, 0.52116651, 0.52116651, 0.52116651,
        0.52116651, 0.52116651, 0.47883349, 0.52116651, 0.52116651,
        0.52116651, 0.52116651, 0.52116651, 0.52116651, 0.52116651,
        0.52116651, 0.52116651, 0.52116651, 0.52116651, 0.52116651,
        0.52116651, 0.52116651, 0.52351834, 0.52351834, 0.52351834,
        0.52351834, 0.52351834, 0.47648166, 0.47648166, 0.52351834,
        0.52351834, 0.52351834, 0.51411101, 0.48588899, 0.51411101,
        0.51411101, 0.51411101, 0.51411101, 0.51411101, 0.51411101,
        0.51411101, 0.51411101, 0.51411101, 0.51411101, 0.51411101,
        0.51411101, 0.51411101, 0.51411101, 0.48588899, 0.51411101,
        0.48588899, 0.48588899, 0.48682973, 0.48588899, 0.48682973,
        0.47883349, 0.47883349, 0.52022578, 0.47883349, 0.47883349,
        0.47883349, 0.52116651, 0.47836312, 0.47

In [46]:
chi_squared_statistic

array([1.13204619e-03, 1.13204619e-03, 1.13204619e-03, 1.13204619e-03,
       1.13204619e-03, 1.13204619e-03, 1.13204619e-03, 1.13204619e-03,
       1.13204619e-03, 1.13204619e-03, 1.13204619e-03, 1.13204619e-03,
       1.13204619e-03, 1.13204619e-03, 1.13204619e-03, 1.13204619e-03,
       1.13204619e-03, 1.13204619e-03, 1.13204619e-03, 1.13204619e-03,
       1.13204619e-03, 1.13204619e-03, 1.12272898e-01, 1.12272898e-01,
       1.12272898e-01, 1.12272898e-01, 1.12272898e-01, 1.12272898e-01,
       1.12272898e-01, 1.12272898e-01, 1.12272898e-01, 1.12272898e-01,
       4.17632442e-02, 4.17632442e-02, 4.17632442e-02, 4.17632442e-02,
       4.17632442e-02, 4.17632442e-02, 4.17632442e-02, 4.17632442e-02,
       4.17632442e-02, 4.17632442e-02, 1.21075488e+00, 1.21075488e+00,
       1.21075488e+00, 1.21075488e+00, 1.21075488e+00, 1.21075488e+00,
       1.21075488e+00, 1.21075488e+00, 1.21075488e+00, 1.21075488e+00,
       1.21075488e+00, 1.21075488e+00, 1.21075488e+00, 1.21075488e+00,
      

In [47]:
is_eq

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False, False,